In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

import sys
sys.path.append('..')

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split

import multiprocessing

from data_augmentaion.data_augmentator import DataAugmentor 

import json

import time

### Define research parameters

In [4]:
regression_task = False
# continuous_features = []
metric = 'accuracy'
test_size_proportion = 0.2
augment_sample = 0.5
metrics = ['f1_weighted', 'accuracy', 'balanced_accuracy', 'precision_weighted', 'recall_weighted', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted'] 

search_pipelines = pipelines.get_classification_pipelines()
search_parameters = models.parameters

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]
name_mapping = {
    'random': 'Random',
    'smote': 'SMOTE',
    'cf_random': 'CF Random',
    'cf_genetic': 'CF Genetic',
}


In [5]:
from sklearn.datasets import make_classification

# Define the parameters of the synthetic dataset
n_samples = 600  # Total number of samples
n_features = 12   # Number of features
n_classes = 3    # Number of classes
class_weights = [0.5, 0.2, 0.3]  # Class imbalance ratio

# Create the imbalanced dataset
X, y = make_classification(n_samples=n_samples, 
                           n_features=n_features,
                           n_informative=3,
                           n_classes=n_classes,
                           n_clusters_per_class=2,
                           weights=class_weights,
                           class_sep=0.5, # 1
                           random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
y = pd.Series(y, name="target")

In [6]:
continuous_features = X.columns.tolist()

#### split data

In [7]:
# all data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

In [8]:
display(y_train.value_counts())

0    250
2    142
1     88
Name: target, dtype: int64

In [9]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [10]:
print(f'whole adult dataset {metric} scores:')
whole_best_ests, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index')
whole_dataset_result_df.columns = pd.MultiIndex.from_product([['whole']] + [whole_dataset_result_df.columns])
whole_dataset_result_df



whole adult dataset accuracy scores:


,whole
,accuracy
lg,0.583333
rf,0.566667
xgb,0.616667


In [11]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df], axis=1)

### running experiments

In [12]:
best_estimators = {}
best_scores = {}
total_time = time.time()
for i, s in enumerate(settings):
    start = time.time()
    print(f'{i} / {len(settings)}, {s}', end=' ')
    augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             cf_scoring = metric,
                             kw_args=s.get('kw_args', {})
                             )

    X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
    best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
     utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
                            search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
    result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'], orient='index')
    result_df_balanced.columns = pd.MultiIndex.from_product([[f'{json.dumps((list(s.values())))} balanced']] + [result_df_balanced.columns])


    X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
    best_estimators[f'{i}'], best_scores[f'{i}'] = \
        utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
    print(best_scores)
    result_df = pd.DataFrame.from_dict(best_scores[f'{i}'], orient='index')
    result_df.columns = pd.MultiIndex.from_product([[f'{json.dumps((list(s.values())))}']] + [result_df.columns])


    full_results_df = pd.concat([full_results_df, result_df], axis=1)
    print(f'{time.time() - start} seconds for settings {i}')
print(f'\nTotal time: {time.time() - total_time}')


0 / 4, {'method': 'random'} 

/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

{'0_balanced': {'lg': {'f1_weighted': 0.5589141511881327, 'accuracy': 0.5583333333333333, 'balanced_accuracy': 0.5427777777777777, 'precision_weighted': 0.5598610694985408, 'recall_weighted': 0.5583333333333333, 'roc_auc_ovr_weighted': 0.7268931878306878, 'roc_auc_ovo_weighted': 0.7193020833333333}, 'rf': {'f1_weighted': 0.5921431407583879, 'accuracy': 0.6, 'balanced_accuracy': 0.5577777777777778, 'precision_weighted': 0.6034382284382285, 'recall_weighted': 0.6, 'roc_auc_ovr_weighted': 0.7721974206349207, 'roc_auc_ovo_weighted': 0.7709861111111111}, 'xgb': {'f1_weighted': 0.5992104957622199, 'accuracy': 0.6, 'balanced_accuracy': 0.5877777777777778, 'precision_weighted': 0.6081349206349207, 'recall_weighted': 0.6, 'roc_auc_ovr_weighted': 0.7651438492063493, 'roc_auc_ovo_weighted': 0.7675868055555555}}, '0': {'lg': {'f1_weighted': 0.5020331300203313, 'accuracy': 0.5833333333333334, 'balanced_accuracy': 0.5083333333333333, 'precision_weighted': 0.46804249390456293, 'recall_weighted': 0.58

100%|██████████| 1/1 [00:00<00:00,  9.18it/s]


50/108


...

150/240


100%|██████████| 1/1 [00:00<00:00, 14.78it/s]


200/240


100%|██████████| 1/1 [00:00<00:00, 14.73it/s]
/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pelegshefi/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

{'0_balanced': {'lg': {'f1_weighted': 0.5589141511881327, 'accuracy': 0.5583333333333333, 'balanced_accuracy': 0.5427777777777777, 'precision_weighted': 0.5598610694985408, 'recall_weighted': 0.5583333333333333, 'roc_auc_ovr_weighted': 0.7268931878306878, 'roc_auc_ovo_weighted': 0.7193020833333333}, 'rf': {'f1_weighted': 0.5921431407583879, 'accuracy': 0.6, 'balanced_accuracy': 0.5577777777777778, 'precision_weighted': 0.6034382284382285, 'recall_weighted': 0.6, 'roc_auc_ovr_weighted': 0.7721974206349207, 'roc_auc_ovo_weighted': 0.7709861111111111}, 'xgb': {'f1_weighted': 0.5992104957622199, 'accuracy': 0.6, 'balanced_accuracy': 0.5877777777777778, 'precision_weighted': 0.6081349206349207, 'recall_weighted': 0.6, 'roc_auc_ovr_weighted': 0.7651438492063493, 'roc_auc_ovo_weighted': 0.7675868055555555}}, '0': {'lg': {'f1_weighted': 0.5020331300203313, 'accuracy': 0.5833333333333334, 'balanced_accuracy': 0.5083333333333333, 'precision_weighted': 0.46804249390456293, 'recall_weighted': 0.58

In [13]:
full_results_df

whole  ["random"]                                                 \
     accuracy f1_weighted  accuracy balanced_accuracy precision_weighted   
lg   0.583333    0.502033  0.583333          0.508333           0.468042   
rf   0.566667    0.518568  0.575000          0.504444           0.647908   
xgb  0.616667    0.620810  0.625000          0.602222           0.632371   

                                                                ["smote"]  \
    recall_weighted roc_auc_ovr_weighted roc_auc_ovo_weighted f1_weighted   
lg         0.583333             0.709737             0.700104    0.535022   
rf         0.575000             0.758631             0.753972    0.584567   
xgb        0.625000             0.777409             0.776844    0.583607   

               ...   ["cf_random"]                                            \
     accuracy  ... recall_weighted roc_auc_ovr_weighted roc_auc_ovo_weighted   
lg   0.533333  ...        0.541667             0.735865             0.728493   
rf   0.583333  ...        0.591667             0.743953             0.736306   
xgb  0.583333  ...        0.616667             0.781300             0.780875   

    ["cf_genetic", {"proximity_weight": 5, "diversity_weight": 0.2, "sparsity_weight": 0.2, "total_CFs": 1, "desired_class": 1}]  \
                                                                                                                     f1_weighted   
lg                                            0.517778                                                                             
rf                                            0.559882                                                                             
xgb                                           0.639195                                                                             

                                                                    \
     accuracy balanced_accuracy precision_weighted recall_weighted   
lg   0.600000          0.525000           0.477731        0.600000   
rf   0.608333          0.547778           0.702412        0.608333   
xgb  0.641667          0.622778           0.654708        0.641667   

                                               
    roc_auc_ovr_weighted roc_auc_ovo_weighted  
lg              0.696093             0.686722  
rf              0.766832             0.760885  
xgb             0.758644             0.778531  

[3 rows x 29 columns]

In [14]:
print('best methods')
best_methods = pd.DataFrame()
for s in metrics:
    models_scores = full_results_df.xs(s, axis='columns', level=1)
    max_models = models_scores.idxmax(axis='columns')
    max_models.name = s
    max_models['overall'] = models_scores.max(axis=0).idxmax(axis=0)
    best_methods = pd.concat([best_methods, max_models], axis=1)
best_methods

best methods


,f1_weighted,accuracy,balanced_accuracy,precision_weighted,recall_weighted,roc_auc_ovr_weighted,roc_auc_ovo_weighted
lg,"[""cf_random""]","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_random""]","[""cf_random""]","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_random""]","[""cf_random""]"
rf,"[""cf_random""]","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""smote""]","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""smote""]"
xgb,"[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_random""]","[""cf_random""]"
overall,"[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_genetic"", {""proximity_weight"": 5, ""divers...","[""cf_random""]","[""cf_random""]"


In [15]:
full_results_df.to_csv(rf'../log/experiment_multiclass_artifical.csv')